<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
import re
from catboost import CatBoostRegressor
from catboost import Pool, cv
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
from pandas_profiling import ProfileReport
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.3


In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [6]:
VERSION    = 16
#DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
#DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
#Kaggle
!ls '../input'
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [7]:
train = pd.read_csv('all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [8]:
print('Размер train', train.shape)
print('Размер test', test.shape)

Размер train (89378, 26)
Размер test (34686, 32)


У нас разное количество признаков на train и  test. Приведем все к единому виду.  

In [9]:
train.sample(5)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
18455,Седан,DAEWOO,DEA522,бензин,2008.0,1.5 MT (80 л.с.),4.0,2010,SEDAN MECHANICAL 1.5,MECHANICAL,1.5,80.0,"Пробег 105000 км,на 85000 км,была замена ремня...",105000,"{'id': '2457167', 'name': 'Базовая', 'availabl...",передний,LEFT,NaN,2.0,ORIGINAL,True,"{'year': 2010, 'month': 8}",130000.0,2019-04-15T15:20:31Z,NaN,NEXIA
49932,Седан,NISSAN,040001,бензин,2008.0,3.5 CVT (249 л.с.),4.0,2008,SEDAN VARIATOR 3.5,VARIATOR,3.5,249.0,Я владею автомобилем с июля 2013 года. Автомоб...,305000,"{'id': '2429048', 'name': 'Premium', 'availabl...",передний,LEFT,NaN,2.0,ORIGINAL,True,"{'year': 2009, 'month': 8}",470000.0,2020-08-23T13:31:26Z,NaN,TEANA
41015,Внедорожник 5 дв.,MAZDA,FAFBFB,бензин,2011.0,2.0 AT (150 л.с.),5.0,2015,ALLROAD_5_DOORS AUTOMATIC 2.0,AUTOMATIC,2.0,150.0,"Машина в хорошем состоянии, повреждения на бам...",51000,{'id': '0'},передний,LEFT,NaN,1.0,ORIGINAL,True,"{'year': 2015, 'month': 2}",1400000.0,2018-03-24T21:22:09Z,NaN,CX_5
66783,Пикап Двойная кабина,TOYOTA,040001,дизель,2011.0,3.0d AT (170 л.с.) 4WD,4.0,2013,PICKUP_TWO AUTOMATIC 3.0d,AUTOMATIC,3.0d,170.0,Полностью подготовленный автомобиль. 35 колеса...,110000,{'id': '0'},полный,LEFT,NaN,1.0,ORIGINAL,True,"{'year': 2013, 'month': 9}",2250000.0,2018-12-20T20:26:31Z,NaN,HILUX
42104,Внедорожник 5 дв.,MITSUBISHI,FAFBFB,бензин,2020.0,1.6 MT (117 л.с.),5.0,2020,ALLROAD_5_DOORS MECHANICAL 1.6,MECHANICAL,1.6,117.0,———————————————————————————\nФУНКЦИОНАЛЬНОЕ ОБ...,0,"{'id': '21907890', 'name': 'Invite', 'availabl...",передний,LEFT,NaN,NaN,ORIGINAL,True,NaN,1548360.0,2020-09-01T07:25:02Z,NaN,ASX


In [10]:
test.sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
25363,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/polo...,белый,NaN,РОЛЬФ Северо-Запад - это первый мегамолл автом...,1.6 LTR,90 N12,"{""engine-proof"":true,""electro-window-back"":tru...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,13932,2014,"{""code"":""POLO"",""name"":""Polo"",""ru_name"":""Поло"",...",POLO,1.6 MT (90 л.с.),4,1603221131,RUB,2017,1100792880,"{""id"":""20726486"",""displacement"":1598,""engine_t...",SEDAN MECHANICAL 1.6,механическая,EUROPEAN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
25823,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,чёрный,"{""id"":""6119123"",""name"":""Comfortline"",""availabl...",Доброго времени суток любители немцев! Продаю ...,1.8 LTR,152 N12,"{""cruise-control"":true,""esp"":true,""airbag-driv...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,221000,2005,"{""code"":""PASSAT"",""name"":""Passat"",""ru_name"":""Па...",PASSAT,1.8 AMT (152 л.с.),4,1603222840,RUB,2010,1096426690,"{""id"":""4553930"",""displacement"":1798,""engine_ty...",SEDAN ROBOT 1.8,роботизированная,EUROPEAN,3 или более,4 года и 1 месяц,Оригинал,передний,Левый,Не требует ремонта,Растаможен
21525,седан,TOYOTA,https://auto.ru/cars/used/sale/toyota/avensis/...,красный,"{""id"":""2427159"",""name"":""Люкс"",""available_optio...",Машина в хорошем техническом состоянии. Есть н...,2.0 LTR,147 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,180950,2006,"{""code"":""AVENSIS"",""name"":""Avensis"",""ru_name"":""...",AVENSIS,2.0 AT (147 л.с.),4,1603295860,RUB,2007,1097123282,"{""id"":""2309539"",""displacement"":1998,""engine_ty...",SEDAN AUTOMATIC 2.0,автоматическая,JAPANESE,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
26883,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/paje...,коричневый,NaN,- Один собственник\n- Оригинал ПТС\n- Автомоби...,3.0 LTR,178 N12,"{""electro-window-back"":true,""esp"":true,""rain-s...",бензин,https://autoru.naydex.net/mcQh17801/f1b5751dCi...,75262,2011,"{""code"":""PAJERO"",""name"":""Pajero"",""ru_name"":""Па...",PAJERO,3.0 AT (178 л.с.) 4WD,5,1603260603,RUB,2013,1101267180,"{""id"":""7744024"",""displacement"":2972,""engine_ty...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,JAPANESE,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
7869,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x5/11012086...,серый,"{""id"":""21640936"",""name"":""xDrive30d M Sport Pro...",Автомобиль продаётся официальным дилером BMW -...,3.0 LTR,249 N12,"{""cruise-control"":true,""sport-seats"":true,""e-a...",дизель,https://autoru.naydex.net/lICN99c69/9bc7d3Rr7k...,4189,2018,"{""code"":""X5"",""name"":""X5"",""ru_name"":""Х5"",""morph...",X5,30d 3.0d AT (249 л.с.) 4WD,5,1603108895,RUB,2020,1101208617,"{""id"":""21308274"",""name"":""30d"",""nameplate"":""30d...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


model_name в test и model в train значат одно и тоже. Переименуем model в train в model_name 

In [11]:
train.rename(columns={'model': 'model_name'}, inplace=True)

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-null  float64
 7   productionDate        89378 non-null  int64  
 8   vehicleConfiguration  89377 non-null  object 
 9   vehicleTransmission   89377 non-null  object 
 10  engineDisplacement    89377 non-null  object 
 11  enginePower           89377 non-null  float64
 12  description           86124 non-null  object 
 13  mileage               89378 non-null  int64  
 14  Комплектация          89378 non-null  object 
 15  Привод             

# EDA

In [14]:
profile = ProfileReport(train, title="Pandas Profiling Report Train")
profile.to_file("PPR_train.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
profile = ProfileReport(test, title="Pandas Profiling Report Test")
profile.to_file("PPR_test.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Сделаем краткие выводы.     
Больше всего тестовом датасете представлены внедорожники и Марседесов.    
Цвета надо будет привести к единой кодировке.    
Почти все машины представлены с 1980 года.    
Судя по milage у нас много новых машин.    
Признак Таможня не информативен. Все значения имеют True.    
hidden тоже не информативен. Все значения пропущены.   
Так же видна сильная корреляция между model date, production date, и отрицательная корреляция с milage.
Так же видим большой дисбаланс классов типов автомобилей и марок.

У нас разное количество признаков в тесте и трейне. Приведем после все к единому виду. Для начала посмотрим на пропуски и посмотрим какие данные у нас предоставлены. 

# Начнем с Train

In [16]:
train.isnull().sum()

bodyType                    1
brand                       0
color                       0
fuelType                    0
modelDate                   1
name                        1
numberOfDoors               1
productionDate              0
vehicleConfiguration        1
vehicleTransmission         1
engineDisplacement          1
enginePower                 1
description              3254
mileage                     0
Комплектация                0
Привод                      1
Руль                        0
Состояние               89378
Владельцы               13588
ПТС                       361
Таможня                     0
Владение                60177
price                     410
start_date                  0
hidden                  89378
model_name                  0
dtype: int64

In [17]:
train[train['bodyType'].isnull()]

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model_name
24624,NaN,HONDA,CACECB,универсал,NaN,NaN,NaN,1989,NaN,NaN,NaN,NaN,"Продаю Honda Civic Suttle 1989 г. выпуска, неу...",400000,"{'id': '0', 'name': ''}",NaN,LEFT,NaN,3.0,ORIGINAL,True,NaN,120000.0,2020-04-10T16:07:57Z,NaN,CIVIC


У нас есть пересечение пропусков, где пропущено всего 1 значение. это civic 1989 года. Удалим эти данные. Они не сильно повлияют на обучение модели. 

In [18]:
train.drop(index=[24624], inplace = True)

description — это текстовое описание рекламы. Оставим столбец так как есть.    
Состояние - в трейне одни пропуски в признаке. в тесте все машины на ходу. Примем,  что все машины в трейне на ходу и удалил признак, т.к. он не информативен.   
Владельцы - если посмотреть на пропуски. То start_date совпадает с productionDate либо есть небольшая разница. Примем, что все пропуски - это новые машины. И поставим значение 0.     
ПТС - опять же, все пропуски только в новых машинах. Заменим значение на 'ORIGINAL'. Так же, данный столбце придеться удалить. Так как у нас все значения равны 'ORIGINAL'. К сожелению в тесте есть значения и дублекат. Однако в связи с тем, что на обучающей выборке у нас таких данных нету, мы не сможем исследовать влияние данного признака на стоимость.    
Владение - столбец содержит информацию, как долго владел авто последний владелец. К сожелению заполнить данный столбец невозможно. Его придеться удалить. Так же на тесте более 60% пропусков у данного признака.    
price - целевая переменная, на которой мы будем обучаться. Удаляем эти столбцы
hidden- одни Nan. Удаляем признак

In [19]:
train.drop('Состояние', axis = 1, inplace = True)
test.drop('Состояние', axis = 1, inplace = True)
train.drop('ПТС', axis = 1, inplace = True)
test.drop('ПТС', axis = 1, inplace = True)
train.drop('Владение', axis = 1, inplace = True)
test.drop('Владение', axis = 1, inplace = True)
train.drop('hidden', axis = 1, inplace = True)

In [20]:
train.drop(train[train['price'].isnull()].index, axis = 0, inplace = True)

In [21]:
train['price'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
89373    False
89374    False
89375    False
89376    False
89377    False
Name: price, Length: 88967, dtype: bool

In [22]:
train['Владельцы'].fillna(0, inplace = True)

Теперь отберем признаки для построения модели, а так же приведем их к единому типу.

In [23]:
# для создания инвертированных словарей
def invert_dict(my_dict):
    inverted_dict = {}
    for (new_key, new_value) in my_dict.items():
        inverted_dict[new_value] = new_key
    return inverted_dict

# для вычисления объема двигателя из тестового DF
def get_engineDisplacement_test(s):
    if (re.search('\d+', s) is not None) == False:
        return 0
    else:
        return s.replace(' LTR','')

# для вычисления мощности двигателя из тестового DF
def get_enginePower_test(s):
    return s.replace(' N12','')


# для вычисления объема двигателя из train0
def get_obiyom_dvigatelya_train0(s):
    s1 = s['engineDisplacement']
    b = False
    if (s1[-1]=='d'):
        s1 = s1[:-1]
        try:
            f = float(s1)
            if f>100:
                b = True
            elif f>10:
                f/=10
            # return f
        except Exception as ex:
            b = True
        if b==False:
            return f
    try:
        f = float(s1)
        if f>10:
            b = True
    except Exception as ex:
        b = True
    if b==False:
        return s1
    else:
        s2 = s['name']
        i = s2.find(s['engineDisplacement']) 
        if i==-1:
            return -1
        else:
            j = s2.find("."[i:])
            s3 = s2[j-1:j+2]
    try:
        f2 = float(s3)
    except Exception as es:
        return -1
    if f2>10:
        return -1
    return f2



# для получения первого слова
def get_first_word(s):
    if not isinstance(s, str):
        return ""
    s = s.strip()
    i = s.find(' ')
    if i<1:
        return s
    else:
        return s[0:i]

In [24]:
# создадим колонку с объемом двигателя (из колонки 'engineDisplacement')
train['engineDisplacement'] = train.apply(get_obiyom_dvigatelya_train0, axis=1).astype(float)
test['engineDisplacement'] = test['engineDisplacement'].apply(get_engineDisplacement_test).astype(float)
# приведем к общему виду кузова автомобилей
test.bodyType = test.bodyType.apply(get_first_word).apply(lambda x: x.upper())
train.bodyType = train.bodyType.apply(get_first_word).apply(lambda x: x.upper())

In [25]:
# словарь для кодирования цветов автомобилей
dict_color_direct       = {'ЗЕЛЁНЫЙ' : 0, 'БЕЛЫЙ' : 1, 'ЧЁРНЫЙ' : 2, 'КОРИЧНЕВЫЙ' : 3, 
                           'СЕРЕБРИСТЫЙ' : 4, 'СЕРЫЙ' : 5, 'КРАСНЫЙ' : 6, 'СИНИЙ' : 7, 'БЕЖЕВЫЙ' : 8, 
                           'ГОЛУБОЙ' : 9, 'ФИОЛЕТОВЫЙ' : 11, 'ЗОЛОТИСТЫЙ' : 12, # 'БОРДОВЫЙ' : 6, # 'Бордовый' : 10, 
                           'ОРАНЖЕВЫЙ' : 13, 'ЖЁЛТЫЙ' : 14, 'ПУРПУРНЫЙ' : 15, 'РОЗОВЫЙ' : 16}
dict_color2_direct      = {'007F00' : 0, 'FAFBFB' : 1, '040001' : 2, '200204' : 3, 'CACECB' : 4, 
                           '97948F' : 5, 'EE1D19' : 6, '0000CC' : 7, 'C49648' : 8, '22A0F8' : 9, 
                           '4A2197' : 11, 'DEA522' : 12, 'FF8649' : 13, 'FFD600' : 14, 
                           '660099' : 15, 'FFC0CB' : 16}
dict_color_invert       = invert_dict(dict_color_direct)

In [26]:
# Преобразование color к единому виду
train['color'] = train['color'].map(dict_color2_direct).map(dict_color_invert)
test.color = test.color.apply(lambda x: x.upper())

In [27]:
# Преобразование fuelType к единому виду
train.color = train.color.apply(lambda x: x.upper())
test.color = test.color.apply(lambda x: x.upper())

In [28]:
# Преобразование modelDate к единому виду
test['modelDate'] = test['modelDate'].astype(int)
train['modelDate'] = train['modelDate'].astype(int)

In [29]:
# Преобразование enginePower к единому виду
test['enginePower'] = test['enginePower'].apply(get_enginePower_test).astype(int)
train['enginePower'] = train['enginePower'].astype(int)

In [30]:
# Преобразование fuelType к единому виду
test.fuelType = test.fuelType.apply(lambda x: x.upper())
train.fuelType = train.fuelType.apply(lambda x: x.upper())

In [31]:
# Преобразование numberOfDoors к единому виду
test.numberOfDoors = test.numberOfDoors.astype(int)
train.numberOfDoors = train.numberOfDoors.astype(int)

In [32]:
# Преобразование даты к единому виду
test['parsing_unixtime'] = test['parsing_unixtime'].apply(
        lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
train['start_date'] = train['start_date'].apply(
        lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d'))

In [33]:
# Преобразование признак Владельцы к единому виду
test['Владельцы'] = test['Владельцы'].apply(lambda x: x[0])
train['Владельцы'] = train['Владельцы'].astype(int)

In [34]:
# Преобразование признак руль к единому виду
dict_rul_direct = {'ЛЕВЫЙ' : 'LEFT', 'ПРАВЫЙ' : 'RIGHT'}
test['Руль'] = test['Руль'].apply(lambda x: x.upper()).map(dict_rul_direct)

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88967 entries, 0 to 89377
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   bodyType              88967 non-null  object        
 1   brand                 88967 non-null  object        
 2   color                 88967 non-null  object        
 3   fuelType              88967 non-null  object        
 4   modelDate             88967 non-null  int32         
 5   name                  88967 non-null  object        
 6   numberOfDoors         88967 non-null  int32         
 7   productionDate        88967 non-null  int64         
 8   vehicleConfiguration  88967 non-null  object        
 9   vehicleTransmission   88967 non-null  object        
 10  engineDisplacement    88967 non-null  float64       
 11  enginePower           88967 non-null  int32         
 12  description           85733 non-null  object        
 13  mileage         

In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              34686 non-null  object 
 1   brand                 34686 non-null  object 
 2   car_url               34686 non-null  object 
 3   color                 34686 non-null  object 
 4   complectation_dict    6418 non-null   object 
 5   description           34686 non-null  object 
 6   engineDisplacement    34686 non-null  float64
 7   enginePower           34686 non-null  int32  
 8   equipment_dict        24690 non-null  object 
 9   fuelType              34686 non-null  object 
 10  image                 34686 non-null  object 
 11  mileage               34686 non-null  int64  
 12  modelDate             34686 non-null  int32  
 13  model_info            34686 non-null  object 
 14  model_name            34686 non-null  object 
 15  name               

In [37]:
train['start_date']

0       2019-10-03
1       2020-09-06
2       2020-09-02
3       2020-08-23
4       2020-09-08
           ...    
89373   2020-02-11
89374   2020-05-30
89375   2020-08-30
89376   2019-05-14
89377   2020-08-10
Name: start_date, Length: 88967, dtype: datetime64[ns]

In [38]:
train.rename(columns={'start_date': 'parsing_unixtime'}, inplace=True)

Отметим, что признак ‘Таможня’ тоже не информативен, так как все машины как в трейне так и в тесте расторможены. 
Подберем признаки для построения модели. 


In [39]:
columns_model = ['bodyType', 'brand','color','description','engineDisplacement','enginePower',
             'fuelType','mileage','modelDate','model_name','numberOfDoors','parsing_unixtime',
             'productionDate','Владельцы','Привод','Руль']

In [40]:
train[columns_model]

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,parsing_unixtime,productionDate,Владельцы,Привод,Руль
0,СЕДАН,AUDI,ЧЁРНЫЙ,"Машина в приличном состоянии ,не гнилая не р...",2.8,174,БЕНЗИН,350000,1990,100,4,2019-10-03,1991,3,полный,LEFT
1,СЕДАН,AUDI,КРАСНЫЙ,Машина в оригинале не гнилая все вопросы по те...,1.8,90,БЕНЗИН,173424,1982,100,4,2020-09-06,1986,3,передний,LEFT
2,УНИВЕРСАЛ,AUDI,СИНИЙ,ПТС Оригинал!\nПолный комплект ключей!\nПо рез...,2.3,136,БЕНЗИН,230000,1988,100,5,2020-09-02,1989,3,полный,LEFT
3,СЕДАН,AUDI,СЕРЕБРИСТЫЙ,NaN,1.8,90,БЕНЗИН,240000,1988,100,4,2020-08-23,1989,3,передний,LEFT
4,СЕДАН,AUDI,ЧЁРНЫЙ,"Машина не гнилая, дыр нет, днище целое, даже в...",2.0,101,БЕНЗИН,300000,1990,100,4,2020-09-08,1991,3,передний,LEFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89373,МИНИВЭН,SSANG_YONG,ЧЁРНЫЙ,СВЕТИТ КРУТИТ РАБОТАЕТ ВСЁ. БЕЗ ТОРГА .В МОСК...,2.7,163,ДИЗЕЛЬ,260000,2004,RODIUS,5,2020-02-11,2006,3,полный,LEFT
89374,МИНИВЭН,SSANG_YONG,СЕРЕБРИСТЫЙ,"Автомобиль на ходу, каждый день в движении.\nС...",2.7,163,ДИЗЕЛЬ,300000,2004,RODIUS,5,2020-05-30,2007,3,полный,LEFT
89375,МИНИВЭН,SSANG_YONG,ЧЁРНЫЙ,"Автомобиль куплен в декабре 2015 г., новым. Вс...",2.0,149,ДИЗЕЛЬ,126000,2013,STAVIC,5,2020-08-30,2014,1,полный,LEFT
89376,МИНИВЭН,SSANG_YONG,СЕРЫЙ,Продаю семейного любимца. Большой вместительны...,2.0,149,ДИЗЕЛЬ,84000,2013,STAVIC,5,2019-05-14,2013,3,полный,LEFT


In [41]:
test[columns_model]

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,parsing_unixtime,productionDate,Владельцы,Привод,Руль
0,ЛИФТБЕК,SKODA,СИНИЙ,"Все автомобили, представленные в продаже, прох...",1.2,105,БЕНЗИН,74000,2013,OCTAVIA,5,2020-10-20,2014,3,передний,LEFT
1,ЛИФТБЕК,SKODA,ЧЁРНЫЙ,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,БЕНЗИН,60563,2017,OCTAVIA,5,2020-10-20,2017,1,передний,LEFT
2,ЛИФТБЕК,SKODA,СЕРЫЙ,"Все автомобили, представленные в продаже, прох...",1.8,152,БЕНЗИН,88000,2013,SUPERB,5,2020-10-20,2014,1,передний,LEFT
3,ЛИФТБЕК,SKODA,КОРИЧНЕВЫЙ,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,БЕНЗИН,95000,2013,OCTAVIA,5,2020-10-20,2014,1,передний,LEFT
4,ЛИФТБЕК,SKODA,БЕЛЫЙ,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,БЕНЗИН,58536,2008,OCTAVIA,5,2020-10-20,2012,1,передний,LEFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34681,СЕДАН,BMW,КОРИЧНЕВЫЙ,Продается BMW 316i в отличном состоянии. Прода...,1.6,136,БЕНЗИН,115000,2011,3ER,4,2020-10-26,2014,3,задний,LEFT
34682,СЕДАН,BMW,ЧЁРНЫЙ,Продаётся отличный автомобиль.,2.0,190,ДИЗЕЛЬ,98000,2016,5ER,4,2020-10-26,2018,1,полный,LEFT
34683,СЕДАН,BMW,СЕРЫЙ,Хорошее состояние,2.5,170,БЕНЗИН,360000,1995,5ER,4,2020-10-26,1997,3,задний,LEFT
34684,ВНЕДОРОЖНИК,BMW,КОРИЧНЕВЫЙ,Автомобиль в идеальном состоянии . Куплен новы...,2.0,184,ДИЗЕЛЬ,90500,2012,X1,5,2020-10-26,2013,2,полный,LEFT


## Data Preprocessing

In [42]:
train

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Владельцы,Таможня,price,parsing_unixtime,model_name
0,СЕДАН,AUDI,ЧЁРНЫЙ,БЕНЗИН,1990,2.8 MT (174 л.с.) 4WD,4,1991,SEDAN MECHANICAL 2.8,MECHANICAL,2.8,174,"Машина в приличном состоянии ,не гнилая не р...",350000,{'id': '0'},полный,LEFT,3,True,200000.0,2019-10-03,100
1,СЕДАН,AUDI,КРАСНЫЙ,БЕНЗИН,1982,1.8 MT (90 л.с.),4,1986,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90,Машина в оригинале не гнилая все вопросы по те...,173424,{'id': '0'},передний,LEFT,3,True,60000.0,2020-09-06,100
2,УНИВЕРСАЛ,AUDI,СИНИЙ,БЕНЗИН,1988,2.3 MT (136 л.с.) 4WD,5,1989,WAGON_5_DOORS MECHANICAL 2.3,MECHANICAL,2.3,136,ПТС Оригинал!\nПолный комплект ключей!\nПо рез...,230000,{'id': '0'},полный,LEFT,3,True,99000.0,2020-09-02,100
3,СЕДАН,AUDI,СЕРЕБРИСТЫЙ,БЕНЗИН,1988,1.8 MT (90 л.с.),4,1989,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90,NaN,240000,{'id': '0'},передний,LEFT,3,True,65000.0,2020-08-23,100
4,СЕДАН,AUDI,ЧЁРНЫЙ,БЕНЗИН,1990,2.0 MT (101 л.с.),4,1991,SEDAN MECHANICAL 2.0,MECHANICAL,2.0,101,"Машина не гнилая, дыр нет, днище целое, даже в...",300000,{'id': '0'},передний,LEFT,3,True,100000.0,2020-09-08,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89373,МИНИВЭН,SSANG_YONG,ЧЁРНЫЙ,ДИЗЕЛЬ,2004,2.7d AT (163 л.с.) 4WD,5,2006,MINIVAN AUTOMATIC 2.7d,AUTOMATIC,2.7,163,СВЕТИТ КРУТИТ РАБОТАЕТ ВСЁ. БЕЗ ТОРГА .В МОСК...,260000,{'id': '0'},полный,LEFT,3,True,450000.0,2020-02-11,RODIUS
89374,МИНИВЭН,SSANG_YONG,СЕРЕБРИСТЫЙ,ДИЗЕЛЬ,2004,2.7d AT (163 л.с.) 4WD,5,2007,MINIVAN AUTOMATIC 2.7d,AUTOMATIC,2.7,163,"Автомобиль на ходу, каждый день в движении.\nС...",300000,{'id': '0'},полный,LEFT,3,True,485000.0,2020-05-30,RODIUS
89375,МИНИВЭН,SSANG_YONG,ЧЁРНЫЙ,ДИЗЕЛЬ,2013,2.0d AT (149 л.с.) 4WD,5,2014,MINIVAN AUTOMATIC 2.0d,AUTOMATIC,2.0,149,"Автомобиль куплен в декабре 2015 г., новым. Вс...",126000,"{'id': '10408082', 'name': 'Luxury', 'availabl...",полный,LEFT,1,True,1090000.0,2020-08-30,STAVIC
89376,МИНИВЭН,SSANG_YONG,СЕРЫЙ,ДИЗЕЛЬ,2013,2.0d AT (149 л.с.) 4WD,5,2013,MINIVAN AUTOMATIC 2.0d,AUTOMATIC,2.0,149,Продаю семейного любимца. Большой вместительны...,84000,"{'id': '10408081', 'name': 'Elegance', 'availa...",полный,LEFT,3,True,1180000.0,2019-05-14,STAVIC


# Подготовка данных к наивной модели.

In [43]:
# для baseline просто возьму пару схожих признаков
columns_naiv = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']
df_train = train[columns_naiv]
df_test = test[columns_naiv]

In [44]:
y = train['price']

## Label Encoding

In [45]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

C:\Users\User\AppData\Local\Temp/ipykernel_2928/1662330491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_2928/1662330491.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0 # помечаем где у нас тест


In [46]:
for colum in ['bodyType', 'brand', 'engineDisplacement']:
    data[colum] = data[colum].astype('category').cat.codes


In [47]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [49]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [50]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 107.32%


# CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [51]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [52]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 17.87%


Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [53]:
np.log(y_train)

45050    13.579788
58064    11.982929
32325    13.527828
44804    14.038654
75985    13.487006
           ...    
6291     14.508657
55137    12.542545
77179    13.122363
865      12.641097
15861    14.403297
Name: price, Length: 71173, dtype: float64

In [54]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [55]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [56]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.49%


Как видим точность возросла до 15.51%. Продолжим дальше

# Futcher engineering 

In [57]:
test_2 = test[columns_model]
train_2 = train[columns_model]

In [58]:
price = train['price']

In [59]:
test_2['sample'] = 1 # помечаем где у нас трейн
train_2['sample'] = 0 # помечаем где у нас тест

data_2 = test_2.append(train_2, sort=False).reset_index(drop=True) # объединяем

C:\Users\User\AppData\Local\Temp/ipykernel_2928/1765247368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_2928/1765247368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2['sample'] = 0 # помечаем где у нас тест


Добавим пару новых признаков:
Комбинация год_месяц. Что бы подобрать курс доллара для каждой машины на дату продажи.
Количество лет машине и количество лет модели.


In [60]:
def to_date(x):
    if type(x) is str:
        return datetime.strptime(x, '%Y-%m-%d')
    else: return x

In [61]:
data_2['parsing_unixtime'] = data_2['parsing_unixtime'].apply(
        lambda x: to_date(x))

In [62]:
data_2['parsing_unixtime'] = data_2['parsing_unixtime'].apply(
        lambda x: to_date(x))
data_2['year_pars'] = data_2['parsing_unixtime'].apply(
        lambda x: str(x.year))
data_2['year_month'] = data_2['parsing_unixtime'].apply(
        lambda x: str(x.year)+"_"+str(+x.month))
data_2['machine_age'] = data_2.apply(
        lambda x: int(x['year_pars'])- x['productionDate'], axis = 1)
data_2['model_age'] = data_2.apply(
        lambda x: int(x['year_pars'])- x['modelDate'], axis = 1)

In [63]:
data_2['month_pars'] = data_2['parsing_unixtime'].apply(
        lambda x: str(x.month))

In [64]:
def description_len (x):
    '''Функция принимает на вход строку
    и возвращает длину строки'''
    
    if type(x) is str:
        return len(x)
    else:
        return 0

In [65]:
data_2['description_len'] = data_2['description'].apply(lambda x: description_len(x))

In [66]:
def seasons (x):
    '''Функция для преобразования в сезоны года'''
    if int(x) in [12,1,2]:
        return 'ЗИМА'
    elif int(x) in [3,4,5]:
        return "ВЕСНА"
    elif int(x) in [6,7,8]:
        return 'ЛЕТО'
    elif int(x) in [9,10,11]:
        return 'ОСЕНЬ'

In [67]:
data_2['season'] = data_2['month_pars'].apply(lambda x: seasons(x))

In [68]:
data_2.head()

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,parsing_unixtime,productionDate,Владельцы,Привод,Руль,sample,year_pars,year_month,machine_age,model_age,month_pars,description_len,season
0,ЛИФТБЕК,SKODA,СИНИЙ,"Все автомобили, представленные в продаже, прох...",1.2,105,БЕНЗИН,74000,2013,OCTAVIA,5,2020-10-20,2014,3,передний,LEFT,1,2020,2020_10,6,7,10,1001,ОСЕНЬ
1,ЛИФТБЕК,SKODA,ЧЁРНЫЙ,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,БЕНЗИН,60563,2017,OCTAVIA,5,2020-10-20,2017,1,передний,LEFT,1,2020,2020_10,3,3,10,2609,ОСЕНЬ
2,ЛИФТБЕК,SKODA,СЕРЫЙ,"Все автомобили, представленные в продаже, прох...",1.8,152,БЕНЗИН,88000,2013,SUPERB,5,2020-10-20,2014,1,передний,LEFT,1,2020,2020_10,6,7,10,1001,ОСЕНЬ
3,ЛИФТБЕК,SKODA,КОРИЧНЕВЫЙ,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,БЕНЗИН,95000,2013,OCTAVIA,5,2020-10-20,2014,1,передний,LEFT,1,2020,2020_10,6,7,10,1676,ОСЕНЬ
4,ЛИФТБЕК,SKODA,БЕЛЫЙ,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,БЕНЗИН,58536,2008,OCTAVIA,5,2020-10-20,2012,1,передний,LEFT,1,2020,2020_10,8,12,10,2365,ОСЕНЬ


In [69]:
col_check = ['bodyType','brand', 'engineDisplacement','enginePower','fuelType', 'model_name','Привод', 'Руль']

In [70]:
# проверим данные на выборс
for i in col_check:
    print(i, data_2[i].unique())

bodyType ['ЛИФТБЕК' 'ВНЕДОРОЖНИК' 'ХЭТЧБЕК' 'СЕДАН' 'КОМПАКТВЭН' 'УНИВЕРСАЛ'
 'ПИКАП' 'КУПЕ' 'КАБРИОЛЕТ' 'МИНИВЭН' 'РОДСТЕР' 'МИКРОВЭН' 'КУПЕ-ХАРДТОП'
 'ФАСТБЕК' 'ТАРГА' 'ЛИМУЗИН' 'СЕДАН-ХАРДТОП' 'ФУРГОН']
brand ['SKODA' 'AUDI' 'HONDA' 'VOLVO' 'BMW' 'NISSAN' 'INFINITI' 'MERCEDES'
 'TOYOTA' 'LEXUS' 'VOLKSWAGEN' 'MITSUBISHI' 'CADILLAC' 'CHERY' 'CHEVROLET'
 'CHRYSLER' 'CITROEN' 'DAEWOO' 'DODGE' 'FORD' 'GEELY' 'HYUNDAI' 'JAGUAR'
 'JEEP' 'KIA' 'MAZDA' 'MINI' 'OPEL' 'PEUGEOT' 'PORSCHE' 'RENAULT' 'SUBARU'
 'SUZUKI' 'GREAT_WALL' 'LAND_ROVER' 'SSANG_YONG']
engineDisplacement [ 1.2  1.6  1.8  2.   1.4  1.3  1.   3.6  1.5  1.9  2.8  1.1  2.5  4.2
  3.   4.   5.9  2.7  3.1  2.4  5.2  3.2  4.1  6.3  2.3  6.   2.2  3.7
  2.9  5.   3.3  2.1  2.6  0.   3.5  1.7  0.7  4.4  4.8  5.4  6.6  4.9
  3.8  3.4  3.9  4.6  5.6  4.5  5.5  6.2  4.7  4.3  5.8  5.3  5.7 -1.
  6.4  8.2  7.   0.8  8.1  6.7  7.5  7.4  6.5  6.1  6.8  8.4  7.3]
enginePower [105 110 152 200 102 150  90 180 220 122  70 140 125  54  86  75 

engineDisplacement содержит значения -1 и 0. Проверим данные выбросы.

In [71]:
data_2[data_2['engineDisplacement'] <= 0]['fuelType'].unique()

array(['ЭЛЕКТРО'], dtype=object)

Все данные автомобили - это электрокары. Заменим все значения -1 на 0 

In [72]:
data_2.loc[(data_2.engineDisplacement < 0), 'engineDisplacement'] = 0

# Добавим среднемесячные курсы валют.

In [73]:
# exchange_rate = pd.read_excel('../input/exchange-rate/exchange_rate.xlsx')
exchange_rate =  pd.read_excel('exchange_rate.xlsx')

In [74]:
exchange_rate = exchange_rate[['year_month','kyrs']]

In [75]:
def add_exchange_rate(x):
    #print(x)
    return exchange_rate[exchange_rate['year_month'] == x]['kyrs'].iloc[0]

In [76]:
data_2['kyrs'] = data_2['year_month'].apply(lambda x:add_exchange_rate(x))

In [77]:
data_2.head()

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,parsing_unixtime,productionDate,Владельцы,Привод,Руль,sample,year_pars,year_month,machine_age,model_age,month_pars,description_len,season,kyrs
0,ЛИФТБЕК,SKODA,СИНИЙ,"Все автомобили, представленные в продаже, прох...",1.2,105,БЕНЗИН,74000,2013,OCTAVIA,5,2020-10-20,2014,3,передний,LEFT,1,2020,2020_10,6,7,10,1001,ОСЕНЬ,77.6635
1,ЛИФТБЕК,SKODA,ЧЁРНЫЙ,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,БЕНЗИН,60563,2017,OCTAVIA,5,2020-10-20,2017,1,передний,LEFT,1,2020,2020_10,3,3,10,2609,ОСЕНЬ,77.6635
2,ЛИФТБЕК,SKODA,СЕРЫЙ,"Все автомобили, представленные в продаже, прох...",1.8,152,БЕНЗИН,88000,2013,SUPERB,5,2020-10-20,2014,1,передний,LEFT,1,2020,2020_10,6,7,10,1001,ОСЕНЬ,77.6635
3,ЛИФТБЕК,SKODA,КОРИЧНЕВЫЙ,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,БЕНЗИН,95000,2013,OCTAVIA,5,2020-10-20,2014,1,передний,LEFT,1,2020,2020_10,6,7,10,1676,ОСЕНЬ,77.6635
4,ЛИФТБЕК,SKODA,БЕЛЫЙ,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,БЕНЗИН,58536,2008,OCTAVIA,5,2020-10-20,2012,1,передний,LEFT,1,2020,2020_10,8,12,10,2365,ОСЕНЬ,77.6635


# Модель №1  CatBoost

In [78]:
col_for_model_tree = ['bodyType','brand','color','engineDisplacement','enginePower','fuelType','mileage',
                'modelDate','model_name','numberOfDoors','productionDate','Владельцы','Привод','Руль',
                'year_pars','machine_age','model_age','description_len','season','kyrs','sample']
col_cat_tree = ['bodyType','brand','color','fuelType','model_name','Привод','Руль',
                'description_len','season']
col_drop_tree = ['description','parsing_unixtime', 'year_month','month_pars']

In [79]:
data_tree = data_2[col_for_model_tree].copy()

In [80]:
# используем label encoding т.к. у нас слишком много категориальных переменных
# при использовании one hot encoding у нас получится слишком большая данных
for colum in col_cat_tree:
    data_tree[colum] = data_tree[colum].astype('category').cat.codes

X_valid = data_tree.query('sample == 1').drop(['sample'], axis=1)
X = data_tree.query('sample == 0').drop(['sample'], axis=1)

In [81]:
y = train['price']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [83]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [84]:
predict_test_Cat_Boost = np.exp(model.predict(X_test))
#predict_submission = np.exp(model.predict(X_sub))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_Cat_Boost))*100:0.2f}%")

Точность модели по метрике MAPE: 12.34%


In [102]:
# Важность признаков
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.get_feature_importance(), index=X_train.columns)
feat_importances.nlargest(40).plot(kind='barh', title='Важность признаков')

<AxesSubplot:title={'center':'Важность признаков'}>

In [103]:
feat_importances.nlargest(40)

enginePower           26.099586
modelDate             11.446357
productionDate        10.730603
brand                  8.076914
model_age              6.988659
machine_age            6.932588
mileage                6.778959
engineDisplacement     6.047390
model_name             4.141527
fuelType               3.228685
bodyType               3.128917
Привод                 2.553604
Владельцы              1.347919
description_len        0.980389
numberOfDoors          0.734039
kyrs                   0.266865
color                  0.237675
Руль                   0.133704
season                 0.080515
year_pars              0.065103
dtype: float64

Как мы видим, наиболее значимым признаком является объём двигателя автомобиля. 
В целом и не удивительно, это самая дорогостоящая часть в автомобиле. 

# Модель №2 AdaBoostRegressor

In [106]:
from sklearn.ensemble import AdaBoostRegressor
ABR = AdaBoostRegressor() 
ABR.fit(X_train, np.log(y_train))
predict_test_RFR = np.exp(ABR.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_RFR))*100:0.2f}%")

Точность модели по метрике MAPE: 28.13%


# Модель №3 DecisionTreeRegressor

In [111]:
from sklearn.tree import DecisionTreeRegressor
DTG = DecisionTreeRegressor()
DTG.fit(X_train,  np.log(y_train))
y_pred = DTG.predict(X_test)
predict_test_DTG = np.exp(y_pred)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_DTG))*100:0.2f}%")

Точность модели по метрике MAPE: 18.54%


# Модель №4 RandomForestRegressor

In [112]:
# При тюнинге гиперпараметров наилучший результат был с max_depth = 25
rf = RandomForestRegressor(max_depth = 25).fit(X_train, np.log(y_train))
predict_test_RFR = np.exp(rf.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_RFR))*100:0.2f}%")

Точность модели по метрике MAPE: 12.48%


# стекинг

In [114]:
from sklearn.ensemble import StackingRegressor
estimators = [('AdaBoostRegressor', ABR),('RFR', rf), ('DecisionTreeRegressor', DTG)]
sr = StackingRegressor(estimators = estimators)
sr.fit(X_train, np.log(y_train))
predict_test_RFR = np.exp(sr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_RFR))*100:0.2f}%")

Точность модели по метрике MAPE: 12.55%


Как мы видим по нашим метрикам, наилучшие результаты метрики MAPE достигаються при использовании модели CatBost.    
CatBost - MAPE: 12.34%    
AdaBoostRegressor - MAPE: 28.13%    
DecisionTreeRegressor - 18.54%    
RandomForestRegressor - MAPE: 12.48%    
Стекинг - MAPE: 12.55%    
сиспользуем CatBost для Submission

# Submission

In [118]:
predict_test_Cat_Boost = np.exp(model.predict(X_valid))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,5.898256e+05
1,1100549428,7.604146e+05
2,1100658222,7.632745e+05
3,1100937408,5.775475e+05
4,1101037972,6.873317e+05
5,1100912634,7.517774e+05
6,1101228730,6.061423e+05
7,1100165896,4.089913e+05
8,1100768262,1.624616e+06
9,1101218501,7.255502e+05


# Вывод по работе.
В процессе работы наилучшей моделью оказался СatBoost. Это модель лучше всего подходит в данных условиях.
К сожалению, в данной версии пакетов использовать СatBoost вместе с кроссвалидацией невозможно.
Возможно, другие методы машинного обучения работают хуже т.к. мы использовали label encoding. Использовать one hot encoding невозможно т.к. получим слишком большой дата сет.    
Возможные идеи для доработки:    
В целом, можно попробовать другие методы машинного обучения, а также поделить стоимость автомобиля на курс.    
Так же можно добавить класс авто в зависимости от цены.   